### Установка пакетов

In [241]:
!pip install pyspark

### Импорт библиотек

In [242]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, trim, regexp_replace, desc
from operator import add
import os

### Создание экземпляра **SparkSession**

In [243]:
spark = SparkSession.builder.appName("Homework_1").getOrCreate()

### Генерация текстового файла

In [244]:
if not os.path.exists('data'):
  os.makedirs('data')

In [245]:
with open("data/words.txt", "w") as f:
  f.writelines([
    'lorem ipsum dolor\n',
    'lorem, ipsum dolor   lorem ipsum dolor\n',
    'lorem ipsum__ dolor lorem ?%@ ipsum dolor lorem ipsum dolor\n',
    'lorem%& ipsum dolor lorem ipsum __dolor lorem ipsum ,dolor lorem ipsum dolor lorem dolorem'
  ])

### Считывание файла

In [246]:
text = spark.read.text("data/words.txt")

In [247]:
text.show()

+--------------------+
|               value|
+--------------------+
|   lorem ipsum dolor|
|lorem, ipsum dolo...|
|lorem ipsum__ dol...|
|lorem%& ipsum dol...|
+--------------------+



### Преобразование датафрейма

In [248]:
def prepare_text(col):
  return explode(split(lower(trim(regexp_replace(col, '[^A-Za-z0-9]', ' '))) , ' '))

words = text.select(prepare_text(text.value).alias('word')).where('word != ""')

In [249]:
words.show()

+-----+
| word|
+-----+
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
|dolor|
|lorem|
|ipsum|
+-----+
only showing top 20 rows



### Подсчет количество каждого слова в тексте

In [250]:
words_count = words.groupBy('word').count()

In [251]:
words_count.show()

+-------+-----+
|   word|count|
+-------+-----+
|  lorem|   11|
|  dolor|   10|
|  ipsum|   10|
|dolorem|    1|
+-------+-----+



### Сортировка слов в порядке убывания их количества в тексте

In [252]:
sorted_words_count = words_count.orderBy(desc('count'))

In [253]:
sorted_words_count.show()

+-------+-----+
|   word|count|
+-------+-----+
|  lorem|   11|
|  dolor|   10|
|  ipsum|   10|
|dolorem|    1|
+-------+-----+



### Получение самого часто встречаемого слова

In [254]:
sorted_words_count.first()

Row(word='lorem', count=11)

### Получение самого редко встречаемого слова

In [255]:
sorted_words_count.tail(1)

[Row(word='dolorem', count=1)]

### Подсчет средней встречаемости слов в тексте

In [256]:
sorted_words_count.groupBy().mean().collect()

[Row(avg(count)=8.0)]

### Подсчет уникального числа слов в тексте


In [257]:
sorted_words_count.count()

4

### Подсчет общего числа слов в тексте

In [258]:
sorted_words_count.agg({'count': 'sum'}).collect()[0][0]

32